### Workflow: 
#### xml file (include core and substituents) -> molecule library (mlib) ->
#### conformer library (clib) (include conformer ensemble (ens)) -> 
#### optimized conformer library (clib) -> ASO and AEIF description

In [1]:
import molli
import molli as ml
ml.visual.configure(bgcolor='white')
from molli import Molecule, MoleculeLibrary
from rdkit import Chem
from rdkit.Chem import AllChem

import msgpack
import attrs
from joblib import delayed, Parallel
from molli.external import openbabel as mob

### 准备好 core.cdxml 和 substituents.cdxml 文件

#### Core file

In [2]:
# Produce core.mol2 files from core.cdxml
core_file_path = "core_CPA.cdxml"
cores = ml.CDXMLFile(core_file_path)
core = cores['1_1']
core_mol2=mob.to_mol2_w_ob(core)
with open('core_CPA.mol2','w') as f:
    f.write(core_mol2)
    print("core_CPA.mol2 file written.")
    

core_CPA.mol2 file written.


In [3]:
mol=ml.Molecule.load_mol2('core_CPA.mol2')
mol.label_atoms('{e}{n1} ')
mol.atoms[25].label='AP1'
mol.atoms[26].label='AP2'

with open('modified_core.mol2', 'w') as f:
    mol.dump_mol2(f)
    
for a in mol.atoms:
    print(a)
# mol

Atom(element=C, isotope=None, label='C1 ', formal_charge=0, formal_spin=0)
Atom(element=C, isotope=None, label='C2 ', formal_charge=0, formal_spin=0)
Atom(element=C, isotope=None, label='C3 ', formal_charge=0, formal_spin=0)
Atom(element=C, isotope=None, label='C4 ', formal_charge=0, formal_spin=0)
Atom(element=C, isotope=None, label='C5 ', formal_charge=0, formal_spin=0)
Atom(element=C, isotope=None, label='C6 ', formal_charge=0, formal_spin=0)
Atom(element=C, isotope=None, label='C7 ', formal_charge=0, formal_spin=0)
Atom(element=C, isotope=None, label='C8 ', formal_charge=0, formal_spin=0)
Atom(element=C, isotope=None, label='C9 ', formal_charge=0, formal_spin=0)
Atom(element=C, isotope=None, label='C10 ', formal_charge=0, formal_spin=0)
Atom(element=C, isotope=None, label='C11 ', formal_charge=0, formal_spin=0)
Atom(element=C, isotope=None, label='C12 ', formal_charge=0, formal_spin=0)
Atom(element=C, isotope=None, label='C13 ', formal_charge=0, formal_spin=0)
Atom(element=C, isoto

In [4]:
# core cpa optimization
mol=ml.Molecule.load_mol2('modified_core.mol2')
mol.add_implicit_hydrogens()

# Set all atomic charges to 0.0 if they're not properly initialized
mol.atomic_charges = [0.0] * len(mol.atoms)

mob.obabel_optimize(
    mol=mol,
    ff = "gaff",
    coord_displace=True #Sometimes necessary to get flat bonds/structures to correctly relax
)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Molecule(name='1_1', formula='C20 H11 Cl2 O4 P1')

### Substituents processing

In [5]:

substituent_file_path = "substituent_CPA.cdxml"
substituents= ml.CDXMLFile(substituent_file_path)


In [6]:

# For substituent molecule
attachment_points_1 = [a for a in substituents['1'].atoms if a.atype == ml.AtomType.AttachmentPoint]
print(attachment_points_1)
attachment_points_2 = [a for a in substituents['2'].atoms if a.atype == ml.AtomType.AttachmentPoint]
print(attachment_points_2)
attachment_points_3 = [a for a in substituents['3'].atoms if a.atype == ml.AtomType.AttachmentPoint]
print(attachment_points_3)

[Atom(element=Unknown, isotope=None, label='AP0', formal_charge=0, formal_spin=0)]
[Atom(element=Unknown, isotope=None, label='AP0', formal_charge=0, formal_spin=0)]
[Atom(element=Unknown, isotope=None, label='AP0', formal_charge=0, formal_spin=0)]


### Combine core and substituents to make full molecules

In [7]:
!molli compile modified_core.mol2 -o P.mlib --overwrite

Matched 1 files for importing.
Importing molecules: 100%|██████████████████████| 1/1 [00:00<00:00, 1138.52it/s]


In [8]:
!molli parse substituent_CPA.cdxml -o substituents.mlib --hadd --overwrite

Parsing substituent_CPA.cdxml: 100%|█████████████| 3/3 [00:00<00:00, 235.69it/s]


In [ ]:
!molli combine P.mlib -s substituents.mlib -a AP1 -a AP2 -m same -o R3P_chiral.mlib --hadd --obopt UFF 1000 1e-4 0.02 --overwrite -n1 -b8

Will create a library of size 3
100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.81s/it]


In [41]:
mlib=ml.MoleculeLibrary("R3P_chiral.mlib", readonly=True)
#This instantiates a MoleculeLibrary to be written to
new_mlib = ml.MoleculeLibrary('R3P_chiral_optimized.mlib', overwrite=True, readonly=False)

# optimized_mols={}
with mlib.reading():
    for name in mlib:
        mol = mlib[name]
        # mol.add_implicit_hydrogens()
        mob.obabel_optimize(
            mol=mol,
            ff = "MMFF94",
            inplace=True,
            coord_displace=True #Sometimes necessary to get flat bonds/structures to correctly relax
        )
        with new_mlib.writing():
            mol.name = name  # Update the molecule's name
            #This serializes the molecule to the MoleculeLibrary using the existing name as a retrieval
            new_mlib[mol.name] = mol
        # optimized_mols.update({name: mol})
        print(f"Optimized molecule: {name}")

Optimized molecule: 1_1_1_1
Optimized molecule: 1_1_2_2
Optimized molecule: 1_1_3_3


In [42]:
%mlib_view R3P_chiral_optimized.mlib 1_1_3_3

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [43]:
##Necessary imports
import molli as ml
from molli.pipeline.crest import CrestDriver

#This is the file the Molecules are retrieved from
source = ml.MoleculeLibrary("R3P_chiral_optimized.mlib", readonly=True)

#This is the file the conformer ensembles calculated will be written to.
destination = ml.ConformerLibrary("R3P.clib", readonly=False)

#This configures the driver, number of processes to use for each worker. Can also indicate how much memory to use.
crest = CrestDriver("crest", nprocs=16)

ml.pipeline.jobmap(
    crest.conformer_search,
    source=source, #Source of molecules
    destination=destination, #Where conformers will be written
    cache_dir="./R3P/conf_cache", #Where final outputs will be written, successful or not!
    scratch_dir="./R3P/scratch_dir", #Scratch Directory where calculations will be run
    n_workers=4, #Number of workers to use. In this case, 4 workers, each with 16 processors as defined in the driver.
    kwargs={
        "method": "gfnff", #GFNFF method to be used
        "temp": 298.15, #Temperature to assume
        "chk_topo": True,#Will check topology
    }, #These are arguments used in the conformer_search function and can be specified directly
    progress=True, #Will print out progress
    verbose = True, #Will print out extra information
)

Starting a molli.pipeline.jobmap_sge calculation:
calculation: CrestDriver.conformer_search
None
     source:  MoleculeLibrary(backend=UkvCollectionBackend('R3P_chiral_optimized.mlib'), n_items=3)
destination:  ConformerLibrary(backend=UkvCollectionBackend('R3P.clib'), n_items=1)
scratch dir:  R3P/scratch_dir
Total number of jobs:        3
Exist in destination:        1
To be computed:              2


Finalizing the calculations: 100%|██████████| 2/2 [00:00<00:00, 29.83it/s]


In [44]:
!molli ls R3P.clib

 0  1_1_3_3  
 1  1_1_2_2  
 2  1_1_1_1  


In [50]:
clib = ml.ConformerLibrary('R3P.clib')
with clib.reading():
    ens = clib['1_1_3_3']
    print(ens)
ens

ConformerEnsemble(name='1_1_3_3', formula='C34 H25 O4 P1', n_conformers=9)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

ConformerEnsemble(name='1_1_3_3', formula='C34 H25 O4 P1')

### Align

In [51]:
!molli grid -s 1.0 -n 32 -b 128 R3P.clib --prune --nearest

Using output file: R3P_grid.hdf5
Computing a bounding box: 100%|███████████████████| 1/1 [00:00<00:00,  1.92it/s]
Finished calculating the bounding box!
Vectors: [-9.177 -6.060 -3.928], [8.846 6.032 4.092]. Number of grid points: 2223. Volume: 1748.05 A**3.
Requested to calculate grid pruning with max_dist=2.000 eps=0.500
Nearest atoms:: 100%|█████████████████████████████| 1/1 [00:00<00:00,  2.90it/s]


### ASO and AEIF calculation

In [52]:
!molli gbca aso R3P.clib -n 64 -b 128

To be computed: 3 ensembles. Skipping 0
Computing descriptor ASO: 100%|███████████████████| 1/1 [00:00<00:00,  1.98it/s]


In [53]:
!molli gbca aeif R3P.clib -n 64 -b 128

To be computed: 3 ensembles. Skipping 0
Computing descriptor AEIF: 100%|██████████████████| 1/1 [00:00<00:00,  2.01it/s]
